In [1]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
from ruamel.yaml import YAML
from sklearn.linear_model import LinearRegression

In [2]:
from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

In [3]:
pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}
yaml = YAML()

kB = 1000 * Boltzmann / electron_volt

In [18]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)
print(fe_structure_4atom)

Full Formula (Fe4)
Reduced Formula: Fe
abc   :   4.053843   4.053843   2.866500
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b     c
---  ----  ---  ---  ----
  0  Fe    0    0     0
  1  Fe    0.5  0.5  -0
  2  Fe    0.5  0     0.5
  3  Fe    1    0.5   0.5


In [5]:
fe_magnetic_patterns = yaml.load(Path("data/fe_magnetic_patterns.yml"))
pd.DataFrame(fe_magnetic_patterns["4atoms"]).style \
    .set_table_styles(html_table_style) \
    .hide_index()

nm,f-type,a-type,g-type
0,1,1,1
0,1,1,-1
0,1,-1,1
0,1,-1,-1


In [6]:
fe_neighbor_data = count_neighbors(cell_structure=fe_structure_4atom, r=3)

In [8]:
fe_neighbor_data

NeighborData(data_frame=   i  j subspecies_i subspecies_j distance_bin                               n
0  0  0  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2
1  0  1  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  8
2  0  2  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
3  0  3  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
4  1  1  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2
5  1  2  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
6  1  3  Fe           Fe           (1.2412309099740464, 2.6744809099740463]  8
7  2  2  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2
8  2  3  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  8
9  3  3  Fe           Fe           (2.6744809099740463, 3.0585190900259525]  2, bins_data_frame=                                         distance_bin                               \
(1.2

In [138]:
x = (np.array(fe_neighbor_data[1]))
nn_distances = (x[:,1]).astype(float)
nn_distances

array([2.48246182, 2.8665    ])

In [224]:
n_sites = int(np.size(fe_structure_4atom))
first_nn_list = np.zeros(n_sites)
first_nn_list = [ (fe_structure_4atom.get_neighbors(fe_structure_4atom[i], r=nn_distances[0])) \
           for i in range(n_sites) ]
first_nn_list[6]

[(PeriodicSite: Fe (10.0328, -1.4333, 12.8992) [0.3750, 0.5000, 1.1250],
  2.4824618199480915),
 (PeriodicSite: Fe (10.0328, -1.4333, 10.0328) [0.3750, 0.5000, 0.8750],
  2.4824618199480915),
 (PeriodicSite: Fe (10.0328, 1.4332, 12.8992) [0.5000, 0.3750, 1.1250],
  2.4824618199480915),
 (PeriodicSite: Fe (10.0328, 1.4332, 10.0328) [0.5000, 0.3750, 0.8750],
  2.4824618199480906)]

In [237]:
list_fe_structure_4atom = list(fe_structure_4atom)
list_fe_structure_4atom[0]
#first_nn_list[6][0]

PeriodicSite: Fe (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

In [215]:
type(list(fe_structure_4atom)), type(first_nn_list)

(list, list)

In [87]:
nn_site =np.array( [ [i for i, j in enumerate(list(fe_structure_4atom)) \
             if j == first_nn_list[n] ] for n in range(n_sites)  ] )

In [252]:
first_nn_list[6]

[(PeriodicSite: Fe (10.0328, -1.4333, 12.8992) [0.3750, 0.5000, 1.1250],
  2.4824618199480915),
 (PeriodicSite: Fe (10.0328, -1.4333, 10.0328) [0.3750, 0.5000, 0.8750],
  2.4824618199480915),
 (PeriodicSite: Fe (10.0328, 1.4332, 12.8992) [0.5000, 0.3750, 1.1250],
  2.4824618199480915),
 (PeriodicSite: Fe (10.0328, 1.4332, 10.0328) [0.5000, 0.3750, 0.8750],
  2.4824618199480906)]